<a href="https://colab.research.google.com/github/silviootero/Proyecto_IA_ForestPrediction/blob/main/02_Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline

In [ ]:
!pip install pyarrow

In [ ]:
!pip install fastparquet

In [ ]:
df = pd.read_parquet('https://mlearner.s3.amazonaws.com/data/raw_data.parquet')
df.head()

In [ ]:
k = df.isna().sum()
k[k!=0]

In [ ]:
list_na = df.columns[df.isna().any()].tolist()
list_na

In [ ]:
for i in list_na:
    df[i].fillna(df[i].mean(),inplace=True)

In [ ]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(df, col):
    assert sum(df[col].isna())==0, "column must have no NaN values"
    values = np.unique(df[col]
                      )
    k = to_onehot(df[col].values)
    r = pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=df.index).join(df)
    del(r[col])
    return r

In [ ]:
list_cat = [i for i in df.columns if not i in df._get_numeric_data()]
list_cat

In [ ]:
print(len(list_cat))

In [ ]:
for i in list_cat:
    df_aux = pd.get_dummies(df[i], prefix=i)
    df.drop(i,inplace=True,axis=1)
    df = df.join(df_aux)

In [ ]:
df.head()

In [ ]:
[i for i in df.columns if not i in df._get_numeric_data()]

In [ ]:

df.to_parquet('clean_data.parquet',index=False)